In [1]:
dir_data = '../data/skin-cancer'
dir_target = './outputs/img_to_be_labeled'

seed = 0
input_shape = (28, 28, 1) 
n_classes = 7
n_random_samples = 100

In [2]:
import sys

sys.path.append('../')

from cnntdanet.utils import seed_all

import os
import json
import labelme
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from glob import glob
from sklearn.model_selection import train_test_split

In [3]:
seed_all(seed)

In [4]:
df = pd.read_csv(os.path.join(dir_data, 'hmnist_28_28_L.csv'))

X = df.drop('label', axis=1).values.reshape((-1, *input_shape))
y = df['label'].values

X = X.astype(np.float32)
X /= 255.0

y = y.astype(np.int64)


In [5]:
# Generating samples to be labeled
train_indices, test_indices = train_test_split(np.arange(len(X)), stratify=y, test_size=0.2, random_state=seed)

X_train, y_train = X[train_indices], y[train_indices]  # Will not be used
X_test, y_test = X[test_indices], y[test_indices]

# Random sampling 'n_random_samples' samples
indices = np.random.choice(test_indices, size=n_random_samples, replace=False)

# Generate images
if not os.path.exists(dir_target):
    os.makedirs(dir_target)

for idx in indices:
    img = X[idx]
    img = 255.0 * img
    img = Image.fromarray(np.uint8(img[:, :, 0]), 'L')
    img.save(f'./{dir_target}/{idx}_{y[idx]}.png')


We used [LabelMe](https://github.com/wkentaro/labelme) for labeling segmentation mask. The following codes generate segmentation masks from json files created by LabelMe.

In [6]:
fpath_list = glob(os.path.join(dir_target, '*.json'))

for fpath in fpath_list:
    json_file = open(fpath)
    json_data = json.load(json_file)
    
    mask = np.zeros((28, 28))
    polygons = json_data.get('shapes')
    for polygon in polygons:
        _mask = labelme.utils.shape_to_mask(
        (json_data['imageHeight'], json_data['imageWidth']), 
        polygon['points'], 
        shape_type=None, line_width = 1, point_size = 1
    )
        mask += _mask
    
    name = fpath.split('/')[-1]
    name = name.split('.')[0]
    np.save(os.path.join(dir_target, f'{name}.npy'), mask)